In [32]:
import csv 
import pandas as pd
import numpy as np
from datetime import datetime 
import re
from unidecode import unidecode

In [33]:
#Diretórios
DIR_ESCOLAS = r'..\arquivos csv\escolas.csv'
DIR_MATERIAL_DIDATICO = r'..\arquivos csv\material_didatico.csv'
DIR_SUB_PREFEITURAS = r'..\arquivos csv\subprefeituras.csv'

# Importações

In [34]:
#Importações
escolas = pd.read_csv(DIR_ESCOLAS)
material_didatico = pd.read_csv(DIR_MATERIAL_DIDATICO)
sub_prefeituras = pd.read_csv(DIR_SUB_PREFEITURAS)

In [35]:
escolas.head(2)

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202"


In [36]:
material_didatico.head(2)

,id,Quantidade
0,178,20
1,634,121


In [37]:
sub_prefeituras.head(2)

,nome,subprefeitura
0,Santo Cristo,Centro
1,Gamboa,Centro


# Merge

In [38]:
#Quantidade de material didático (md) à ser entregue
escolas_quantidade_md = pd.merge(escolas, material_didatico, how='left', on='id')
escolas_quantidade_md.head(2)

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,Quantidade
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326",20
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202",121


In [39]:
#Define os tipos de escolas

def define_tipo(x):
    if 'EM' in x.upper() or 'E.M' in x.upper():
        return 'EM'
    elif 'CIEP' in x.upper() or 'CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA' in x.upper():
        return 'CIEP'
    elif 'ESCOLA' in x.upper():
        return 'ESCOLA'
    else:
        None
    
escolas_quantidade_md['tipo'] = escolas_quantidade_md['Escolas_Postos'].apply(lambda x: define_tipo(x))
escolas_quantidade_md.head(2)

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,Quantidade,tipo
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326",20,CIEP
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202",121,EM


# Tratamentos

In [40]:
# Converte todos os valores strings para UPPER CASE
escolas_quantidade_md = escolas_quantidade_md.applymap(lambda x: x.upper() if isinstance(x, str) else x)
# Remove acentuações
escolas_quantidade_md = escolas_quantidade_md.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
escolas_quantidade_md.head(2)

C:\Users\Danie\AppData\Local\Temp\ipykernel_8940\3799985094.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  escolas_quantidade_md = escolas_quantidade_md.applymap(lambda x: x.upper() if isinstance(x, str) else x)
C:\Users\Danie\AppData\Local\Temp\ipykernel_8940\3799985094.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  escolas_quantidade_md = escolas_quantidade_md.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)


,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,Quantidade,tipo
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,"-22,880888","-43,225326",20,CIEP
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202",121,EM


In [41]:
# Acesse o nome da coluna na posição 4 (índice 3)
coluna_na_posicao_4 = escolas_quantidade_md.columns[3]

# Imprima o nome da coluna na posição 3
print(coluna_na_posicao_4) # indica que "ENDEREÇO" é "ENDEREÇO "

ENDEREÇO 


In [42]:
# Função para substituir palavras em texto da coluna 'ENDEREÇO '
def substituir_palavras(texto, substituicoes):
    for palavra, substituta in substituicoes.items():
        texto = texto.replace(palavra, substituta)
    return texto

substituicoes_endereco = {
    '.': '',
    ',': '',
    'ESTR ': 'ESTRADA ',
    'JR ': 'JUNIOR ',
    'AV ': 'AVENIDA ',
    'MAL ': 'MARECHAL ',
    'ENG ': 'ENGENHEIRO ',
    'SD ': 'SOLDADO ',
    'PCA ': 'PRACA ',
    'ALM ': 'ALMIRANTE ',
    'SEN ': 'SENADOR ',
    'CONJ ': 'CONJUNTO ',
}

# Define um padrão para encontrar "R" como uma palavra independente em endereco para substituição
padrao_r = r'\bR\b'

# Aplica as funções de substituição à coluna endereco
escolas_quantidade_md['ENDEREÇO '] = escolas_quantidade_md['ENDEREÇO '].apply(substituir_palavras, args=(substituicoes_endereco,))
escolas_quantidade_md['ENDEREÇO '] = escolas_quantidade_md['ENDEREÇO '].str.replace(padrao_r, 'RUA', regex=True)

# Função para extrair complementos do endereço
def extrair_complemento(endereco):
    complementos = re.findall(r'\b(BLOCO|CONJUNTO)\s+(\S+)\b', endereco, flags=re.IGNORECASE)
    if complementos:
        return ','.join([f'{comp[0]} {comp[1]}' for comp in complementos])
    else:
        return "SEM COMPLEMENTO"

# Aplica a função de extração e cria uma nova coluna 'complemento'
escolas_quantidade_md['complemento'] = escolas_quantidade_md['ENDEREÇO '].apply(extrair_complemento)

# Função para extrair números do endereço
def extrair_numero(endereco):
    numeros_hifen = re.findall(r'\b\d+-\d+\b', endereco)
    numeros_km = re.findall(r'KM\s*\d+',endereco)
    padrao_sem_numeros = re.findall(r'S/No|S/Ndeg|S/N', endereco)

    if numeros_hifen:
        return '-'.join(numeros_hifen)
    elif numeros_km:
        return numeros_km[-1]
    elif padrao_sem_numeros:
        return 0
    else:
        numeros = re.findall(r'\d+', endereco)
        return int(numeros[-1]) if numeros else 0

# Aplica a função de extração e cria uma nova coluna 'numero_rua'
escolas_quantidade_md['numero_rua'] = escolas_quantidade_md['ENDEREÇO '].apply(extrair_numero)

# Função para remover números e variações do endereço
def remover_numeros_variacoes(endereco, numeros_rua):
    if isinstance(numeros_rua, str):
        endereco = endereco.replace(numeros_rua, '')
    padrao = r'\d+$|S/No|S/Ndeg|S/N|-'
    endereco_limpo = re.sub(padrao, '', endereco).strip()
    return endereco_limpo

# Aplica a função de remoção de números do endereço
escolas_quantidade_md['ENDEREÇO '] = escolas_quantidade_md.apply(lambda row: remover_numeros_variacoes(row['ENDEREÇO '], row['numero_rua']), axis=1)
escolas_quantidade_md.head()

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,Quantidade,tipo,complemento,numero_rua
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,"-22,880888","-43,225326",20,CIEP,SEM COMPLEMENTO,0
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO,"-22,889574","-43,236202",121,EM,SEM COMPLEMENTO,187
2,483,EM CELESTINO SILVA,CENTRO,RUA DO LAVRADIO,"-22,909293","-43,183579",220,EM,SEM COMPLEMENTO,56
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA,"-22,897629","-43,227456",190,ESCOLA,SEM COMPLEMENTO,20
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN,"-22,924412","-43,208579",67,EM,SEM COMPLEMENTO,45


# Subprefeituras


In [43]:
# Converte todos os valores strings para UPPER CASE
sub_prefeituras_md = sub_prefeituras.applymap(lambda x: x.upper() if isinstance(x, str) else x)
sub_prefeituras_md

# Remove acentuações
sub_prefeituras_md = sub_prefeituras_md.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
sub_prefeituras_md

# Teste
escolas_quantidade_md = escolas_quantidade_md.merge(sub_prefeituras_md, left_on='BAIRRO', right_on='nome', how='left')
escolas_quantidade_md.drop(columns=['nome'], inplace=True)
escolas_quantidade_md


C:\Users\Danie\AppData\Local\Temp\ipykernel_8940\1392222361.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sub_prefeituras_md = sub_prefeituras.applymap(lambda x: x.upper() if isinstance(x, str) else x)
C:\Users\Danie\AppData\Local\Temp\ipykernel_8940\1392222361.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sub_prefeituras_md = sub_prefeituras_md.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)


,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,Quantidade,tipo,complemento,numero_rua,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,"-22,880888","-43,225326",20,CIEP,SEM COMPLEMENTO,0,CENTRO
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO,"-22,889574","-43,236202",121,EM,SEM COMPLEMENTO,187,CENTRO
2,483,EM CELESTINO SILVA,CENTRO,RUA DO LAVRADIO,"-22,909293","-43,183579",220,EM,SEM COMPLEMENTO,56,CENTRO
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA,"-22,897629","-43,227456",190,ESCOLA,SEM COMPLEMENTO,20,CENTRO
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN,"-22,924412","-43,208579",67,EM,SEM COMPLEMENTO,45,CENTRO
...,...,...,...,...,...,...,...,...,...,...,...
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA,"-22,98046","-43,643545",320,EM,SEM COMPLEMENTO,9183,ZONA OESTE
148,301,EM JONATAS SERRANO,GUARATIBA,ESTRADA DO MATO ALTO,"-22,953163","-43,577409",335,EM,SEM COMPLEMENTO,0,ZONA OESTE
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO,"-23,009084","-43,537582",NaN,EM,SEM COMPLEMENTO,0,ZONA OESTE
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,CAMINHO DA MATRIZ,"-22,994124","-43,593683",521,EM,SEM COMPLEMENTO,4406,ZONA OESTE


In [44]:
# Renomear as colunas do DataFrame
escolas_quantidade_md.rename(columns={
    'id': 'id',
    'Escolas_Postos': 'nome',
    'BAIRRO': 'bairro',
    'ENDEREÇO ': 'logradouro',
    'lat': 'latitude',
    'lon': 'longitude',
    'Quantidade': 'quantidade_material',
}, inplace=True)

# Reorganiza as colunas do DataFrame
escolas_quantidade_md = escolas_quantidade_md[['id', 'nome', 'tipo', 'logradouro', 'numero_rua', 'complemento', 'latitude', 'longitude', 'bairro', 'subprefeitura' ]]
escolas_quantidade_md.head()

,id,nome,tipo,logradouro,numero_rua,complemento,latitude,longitude,bairro,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CIEP,RUA CARLOS SEIDL,0,SEM COMPLEMENTO,"-22,880888","-43,225326",CAJU,CENTRO
1,634,EM ALICE DO AMARAL PEIXOTO,EM,RUA EBANO,187,SEM COMPLEMENTO,"-22,889574","-43,236202",BENFICA,CENTRO
2,483,EM CELESTINO SILVA,EM,RUA DO LAVRADIO,56,SEM COMPLEMENTO,"-22,909293","-43,183579",CENTRO,CENTRO
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,ESCOLA,PRACA ARGENTINA,20,SEM COMPLEMENTO,"-22,897629","-43,227456",SAO CRISTOVAO,CENTRO
4,132,EM PEREIRA PASSOS,EM,PRACA CONDESSA PAULO DE FRONTIN,45,SEM COMPLEMENTO,"-22,924412","-43,208579",RIO COMPRIDO,CENTRO
